Serial Schedule Generation Scheme (SGS)

The solver iteratively schedules jobs that are ready (all predecessors completed). For each ready job, it determines the earliest feasible start time by considering:

* Precedence Constraints: Must start after all predecessors finish.
* Resource Constraints: Must start when sufficient resources are available for its entire duration.
* Latest Finish Time (LFT) Heuristic: Jobs are prioritized based on their LFT to address potential bottlenecks.

The algorithm maintains a timeline to track resource usage over time and incrementally finds the first available time slot that satisfies all constraints. For non-renewable resources, it tracks total consumption.

Heuristics Employed:

* Job Prioritization (LFT): Jobs are sorted by their calculated Latest Finish Time to address critical path activities first.
* Earliest Feasible Start Time: The solver searches for the absolute earliest time a job can begin by incrementally checking time steps from its earliest possible start time until resource availability is confirmed.
* Resource Type Handling: Differentiates between renewable (capacity) and non-renewable (stock) resources for constraint checking and remaining stock calculation.

In [4]:
import json
import time
from typing import Dict, Any, List

def solve_project(json_data: Dict) -> Dict:
    """
    Solves the RCPSP/JIT problem and calculates remaining resource stocks/capacities.
    """
    start_wall_time = time.time()

    # --- 1. DATA RESOLUTION ---
    # Extract locations from meta structure
    meta = json_data.get('meta', {})
    root = meta.get('root_structure', {})
    job_path = root.get('jobs', 'jobs')
    res_path = root.get('resources', 'resources')
    
    raw_jobs = json_data.get(job_path, [])
    raw_resources = json_data.get(res_path, [])

    # Map Resources
    # We store type to distinguish between Renewable (capacity) and Non-Renewable (stock)
    resource_info = {}
    for res in raw_resources:
        r_id = str(res.get('id'))
        r_type = res.get('type', 'renewable')
        
        if r_type == 'nonrenewable':
            limit = res.get('initial_stock', 0)
        else:
            limit = res.get('capacity', 0)
            
        resource_info[r_id] = {
            'limit': limit,
            'type': r_type,
            'total_consumed': 0
        }

    # Map Jobs
    activities = []
    for job in raw_jobs:
        job_id = job.get('id')
        duration = job.get('duration', 0)
        # Handle nested path: precedences.time_successors
        precedences = job.get('precedences', {})
        successors = precedences.get('time_successors', [])
        requirements = job.get('resources_required', {})
        
        activities.append({
            'id': job_id,
            'duration': duration,
            'demand': {str(k): v for k, v in requirements.items()},
            'successors': successors
        })

    # --- 2. PRE-PROCESSING ---
    act_ids = [a['id'] for a in activities]
    duration_map = {a['id']: a['duration'] for a in activities}
    demand_map = {a['id']: a['demand'] for a in activities}
    succ_map = {a['id']: a['successors'] for a in activities}
    
    pred_map = {aid: [] for aid in act_ids}
    for aid, successors in succ_map.items():
        for s in successors:
            if s in pred_map:
                pred_map[s].append(aid)

    # Heuristic: Latest Finish Time (LFT)
    horizon = sum(duration_map.values()) + 1
    lft = {aid: horizon for aid in act_ids}
    for _ in range(len(act_ids)):
        for aid in act_ids:
            if succ_map[aid]:
                lft[aid] = min(lft[s] - duration_map[s] for s in succ_map[aid] if s in lft)

    # --- 3. SCHEDULING (SERIAL SGS) ---
    scheduled_start = {}
    scheduled_finish = {}
    usage_timeline = {} # t -> {res_id: usage}

    # Sort by LFT to prioritize bottleneck jobs
    unscheduled = sorted(act_ids, key=lambda x: lft[x])

    while len(scheduled_start) < len(act_ids):
        # Find jobs whose predecessors are finished
        eligible = [aid for aid in unscheduled if aid not in scheduled_start 
                    and all(p in scheduled_finish for p in pred_map[aid])]
        
        curr_id = eligible[0]
        dur = duration_map[curr_id]
        reqs = demand_map[curr_id]
        
        # Determine earliest start (Precedence + Renewable Resource Availability)
        t_start = max([scheduled_finish[p] for p in pred_map[curr_id]], default=0)
        
        while True:
            is_feasible = True
            for t in range(t_start, t_start + dur):
                for r_id, amount in reqs.items():
                    info = resource_info.get(r_id, {})
                    # Renewable check
                    if info.get('type') == 'renewable':
                        current_t_usage = usage_timeline.get(t, {}).get(r_id, 0)
                        if current_t_usage + amount > info['limit']:
                            is_feasible = False; break
                if not is_feasible: break
            
            if is_feasible: break
            t_start += 1

        # Commit to schedule
        scheduled_start[curr_id] = t_start
        scheduled_finish[curr_id] = t_start + dur
        
        # Update usage profile and consumption
        for r_id, amount in reqs.items():
            # Update Non-Renewable total consumption
            resource_info[r_id]['total_consumed'] += amount
            # Update timeline for Renewable checks
            for t in range(t_start, t_start + dur):
                if t not in usage_timeline: usage_timeline[t] = {}
                usage_timeline[t][r_id] = usage_timeline[t].get(r_id, 0) + amount

    # --- 4. RESOURCE REMAINING CALCULATION ---
    remaining_resources = {}
    for r_id, info in resource_info.items():
        if info['type'] == 'nonrenewable':
            # For non-renewable, it's the stock left
            remaining_resources[r_id] = info['limit'] - info['total_consumed']
        else:
            # For renewable, it shows total capacity (as it renews)
            # but we could also calculate the 'Idle Capacity Rate'
            remaining_resources[r_id] = info['limit']

    # --- 5. OUTPUT ---
    makespan = max(scheduled_finish.values()) if scheduled_finish else 0
    execution_time = time.time() - start_wall_time

    return {
        "schedule": scheduled_start,
        "remaining_resources": remaining_resources,
        "metrics": {
            "makespan": makespan,
            "total_penalty": 0,
            "execution_time": round(execution_time, 6)
        }
    }

In [6]:
with open("C:/Users/Admin/Desktop/renewable.json", 'r') as f:
    data = json.load(f)  

result = solve_project(data)
print(result)

{'schedule': {0: 0, 1: 0, 49: 0, 25: 0, 37: 0, 2: 0, 3: 0, 4: 0, 5: 1, 6: 2, 13: 0, 7: 1, 15: 0, 38: 0, 50: 0, 51: 0, 52: 0, 8: 7, 9: 7, 26: 0, 39: 0, 42: 0, 27: 0, 29: 0, 14: 0, 28: 1, 30: 8, 31: 9, 55: 0, 41: 9, 40: 12, 43: 10, 16: 5, 17: 0, 18: 5, 19: 0, 53: 10, 54: 10, 10: 15, 11: 15, 12: 24, 20: 14, 21: 7, 22: 7, 23: 0, 24: 15, 32: 14, 33: 12, 34: 16, 35: 17, 36: 26, 44: 18, 45: 20, 46: 20, 47: 23, 48: 33, 56: 19, 57: 10, 58: 12, 59: 19, 60: 29, 61: 33, 62: 33, 74: 0, 63: 33, 64: 33, 75: 8, 77: 8, 65: 39, 76: 14, 67: 39, 68: 33, 66: 39, 78: 15, 79: 15, 69: 48, 70: 48, 71: 44, 72: 44, 73: 54, 80: 24, 81: 17, 82: 8, 83: 11, 84: 21, 85: 29, 86: 54, 87: 54, 90: 54, 91: 54, 88: 54, 89: 56, 92: 58, 93: 68, 94: 64, 95: 58, 96: 64, 97: 74, 98: 74, 99: 74, 100: 77, 101: 74, 102: 74, 103: 84, 104: 84, 105: 82, 106: 85, 107: 83, 108: 74, 109: 93, 110: 93, 111: 93, 112: 93, 113: 93, 114: 103, 115: 99, 116: 112, 117: 104, 118: 99, 119: 106, 120: 103, 121: 120}, 'remaining_resources': {'1': 10,